In [1]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

tf.random.set_seed(0)
np.random.seed(0)

print("Tensorflow version: ", tf.__version__)

Tensorflow version:  2.4.1


In [2]:
gdd.download_file_from_google_drive(file_id='1wjuPDgCDThA2vk-2ctpTxr1HMoYYnm7h', dest_path='./Assignment3.zip', unzip=True)

In [3]:
data_dir = 'Assignment3'
os.listdir(data_dir)

['images', 'sample_submission.csv', 'train.csv']

In [4]:
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

,image,label
0,VietAI-Assignment3-1.jpg,7
1,VietAI-Assignment3-100.jpg,2
2,VietAI-Assignment3-10000.jpg,1
3,VietAI-Assignment3-10001.jpg,2
4,VietAI-Assignment3-10002.jpg,2


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8234 entries, 0 to 8233
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   8234 non-null   object
 1   label   8234 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 128.8+ KB


In [6]:
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
test_df.head()

,image,label
0,VietAI-Assignment3-10.jpg,0
1,VietAI-Assignment3-1000.jpg,0
2,VietAI-Assignment3-10004.jpg,0
3,VietAI-Assignment3-10006.jpg,0
4,VietAI-Assignment3-10012.jpg,0


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   2059 non-null   object
 1   label   2059 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 32.3+ KB


In [8]:
train_df.label.value_counts()

2     1949
1      846
0      808
3      764
5      642
4      599
7      579
10     575
6      535
8      469
9      468
Name: label, dtype: int64

In [9]:
import cv2
def show_gallery(df, n=5, shuffle=True):
  plt.subplots(figsize=(20,20))
  if shuffle:
    df = df.sample(frac=1).reset_index(drop=True)
  k=1
  for i in range(n*n):
    im = cv2.imread(os.path.join("./Assignment3/images",df.loc[k,"image"]))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    label = df.loc[k,"label"]
    plt.subplot(n,n,k)
    plt.imshow(im)
    plt.title("Label: {}".format(label))
    plt.axis("off")
    k += 1

show_gallery(train_df)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
def generate_data(image_paths, size=224):
    """
    Đọc và chuyển các ảnh về numpy array
    
    Parameters
    ----------
    image_paths: list of N strings
        List các đường dẫn ảnh
    size: int
        Kích thước ảnh cần resize
    
    Returns
    -------
    numpy array kích thước (N, size, size, 3)
    """
    image_array = np.zeros((len(image_paths), size, size, 3), dtype='uint8')
    
    for idx, image_path in tqdm(enumerate(image_paths)):
        ### START CODE HERE
        
        # Đọc ảnh bằng thư viện Pillow và resize ảnh
        image = Image.open(image_path)
        newsize = (size, size)
        image = image.resize(newsize)
        
        # Chuyển ảnh thành numpy array và gán lại mảng image_array
        image = np.array(image)
        image_array[idx] = image
        
        
        ### END CODE HERE
    return image_array

In [11]:
# List các đường dẫn file cho việc huấn luyện
train_files = [os.path.join("Assignment3/images", file) for file in train_df.image]

# List các nhãn
train_y = train_df.label

# Tạo numpy array cho dữ liệu huấn luyện
train_arr = generate_data(train_files)

8234it [00:49, 167.35it/s]


In [12]:
test_files = [os.path.join("Assignment3/images", file) for file in test_df.image]
test_x = generate_data(test_files)
test_x.shape

2059it [00:12, 166.86it/s]


(2059, 224, 224, 3)

In [14]:
num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

In [15]:
x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(train_arr, y_ohe, test_size=0.2)
#x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(x_train, y_train_ohe, test_size = 0.2)
print("Train size: {} - Validation size: {}".format(x_train.shape, x_valid.shape))

Train size: (6587, 224, 224, 3) - Validation size: (1647, 224, 224, 3)


In [ ]:
class ResBlock(tf.keras.Model):
  def __init__(self, filters, reps, strides):
    super(ResBlock, self).__init__()
    
    self.reps = reps

    self.batch_norm = tf.keras.layers.BatchNormalization(axis = 3)
    self.activation = tf.keras.layers.ReLU()
    self.add = tf.keras.layers.Add()
    
    # Projection Block
    self.conv2d_1 = tf.keras.layers.Conv2D(filters = filters, kernel_size = 1, strides = strides, padding = 'same')
    self.conv2d_2 = tf.keras.layers.Conv2D(filters = filters, kernel_size = 3, strides = 1, padding = 'same')
    self.conv2d_3 = tf.keras.layers.Conv2D(filters=4*filters, kernel_size=1, strides=1)
    self.conv2d_4 = tf.keras.layers.Conv2D(filters=4*filters, kernel_size=1, strides=strides)

    # Identity Block
    self.conv2d_5 = tf.keras.layers.Conv2D(filters = filters, kernel_size = 1, strides = 1, padding = 'same')
    self.conv2d_6 = tf.keras.layers.Conv2D(filters = filters, kernel_size = 3, strides = 1, padding = 'same')
    self.conv2d_7 = tf.keras.layers.Conv2D(filters=4*filters, kernel_size=1, strides=1)


    #self.projection = ProjectionBlock(filters = filters, strides = strides)
    #self.identities = [IdentityBlock(filters = filters) for i in range(reps - 1)]
    
  def call(self, inputs):
    x = inputs

    # Projection Block
    # leftstream
    
    #block 1
    x = self.conv2d_1(x)
    #print(1)
    x = self.batch_norm(x)
    x = self.activation(x)
    # block 2
    x = self.conv2d_2(x)
    #print(2)
    x = self.batch_norm(x)
    x = self.activation(x)
    # block 3
    x = self.conv2d_3(x)
    
    #print(3)
    ### bugs ###
    #x = self.batch_norm(x)
    #### bugs ### 

    # rightstream
    shortcut = self.conv2d_4(inputs)
    #print(4)
    #shortcut = self.batch_norm(shortcut)
    
    x = self.add([shortcut, x])
    x = self.activation(x)

    # Identity Block
    for _ in range(self.reps - 1):
      y = x
      # block 1
      x = self.conv2d_5(x)
      #print(5)
      x = self.batch_norm(x)
      x = self.activation(x)
      # block 2
      x = self.conv2d_6(x)
      #print(6)
      x = self.batch_norm(x)
      x = self.activation(x)
      # block 3
      x = self.conv2d_7(x)
      #print(7)
      #x = self.batch_norm(x)
      # skip connection
      x = self.add([y, x])
      x = self.activation(x)
    return x

In [ ]:
class ResNet(tf.keras.Model):
  def __init__(self, num_classes):
    super(ResNet, self).__init__()
    
    self.conv2d = tf.keras.layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, padding = 'same')
    self.batch_norm = tf.keras.layers.BatchNormalization()
    self.activation = tf.keras.layers.ReLU()
    self.max_pool = tf.keras.layers.MaxPool2D(pool_size = 3, strides = 2)
    
    self.resnet_1 = ResBlock(filters = 64, reps = 3, strides = 1)
    self.resnet_2 = ResBlock(filters = 128, reps = 4, strides = 2)
    self.resnet_3 = ResBlock(filters = 256, reps = 6, strides = 2)
    self.resnet_4 = ResBlock(filters = 512, reps = 3, strides = 2)
    
    
    self.global_avg = tf.keras.layers.GlobalAvgPool2D()
    
    # Fully connected
    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(units = num_classes, activation = 'softmax')

  def call(self, inputs):
    
    x = inputs / 255
    
    x = self.conv2d(x)
    x = self.batch_norm(x)
    x = self.activation(x)
    x = self.max_pool(x)
    
    x = self.resnet_1(x)
    x = self.resnet_2(x)
    x = self.resnet_3(x)
    x = self.resnet_4(x)
    
    x = self.global_avg(x)
    x = self.flatten(x)

    output = self.dense(x)

    return output

In [16]:
base_model = tf.keras.applications.VGG16(include_top = False, input_shape= (224, 224, 3), classes = num_classes, weights = None)

In [17]:
x = base_model.output
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(num_classes, activation = 'softmax')(x)
model = tf.keras.models.Model(inputs = base_model.input, outputs = output)

In [ ]:
device = '/cpu:0' if len(tf.config.experimental.list_physical_devices('GPU')) == 0 else '/gpu:0'
print(device)
batch_size = 32
epochs = 50

with tf.device(device):
    # Khởi tạo model
    #model = ResNet(num_classes)
    
    # Tạo callback để lưu model có accuracy trên tập validation tốt nhất
    mcp = tf.keras.callbacks.ModelCheckpoint("my_model.h5", monitor="val_accuracy",
                      save_best_only=True, save_weights_only=True)
    
    # Compile model
    model.compile(optimizer=tf.optimizers.SGD(learning_rate = 0.001, momentum= 0.9), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Huấn luyện
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_valid, y_valid_ohe), verbose=1, callbacks=[mcp])


/gpu:0
Epoch 1/50
206/206 [==============================] - 95s 432ms/step - loss: 2.3048 - accuracy: 0.2177 - val_loss: 2.1631 - val_accuracy: 0.2410
Epoch 2/50
206/206 [==============================] - 91s 444ms/step - loss: 2.1673 - accuracy: 0.2590 - val_loss: 2.0818 - val_accuracy: 0.2738
Epoch 3/50
206/206 [==============================] - 92s 445ms/step - loss: 2.0770 - accuracy: 0.2742 - val_loss: 1.9795 - val_accuracy: 0.3218
Epoch 4/50
206/206 [==============================] - 92s 445ms/step - loss: 2.0081 - accuracy: 0.3026 - val_loss: 1.9382 - val_accuracy: 0.3352
Epoch 5/50
206/206 [==============================] - 92s 445ms/step - loss: 1.9389 - accuracy: 0.3282 - val_loss: 1.9477 - val_accuracy: 0.3315
Epoch 6/50
206/206 [==============================] - 92s 444ms/step - loss: 1.8903 - accuracy: 0.3414 - val_loss: 1.9600 - val_accuracy: 0.3230
Epoch 7/50
206/206 [==============================] - 91s 444ms/step - loss: 1.8523 - accuracy: 0.3623 - val_loss: 1.8869 -

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

***Load Model***

In [20]:
model_file = "my_model256.h5"
loaded_model = tf.keras.models.Model(inputs = base_model.input, outputs = output)
loaded_model.load_weights(model_file)

In [21]:
predictions = loaded_model.predict(x_test)
predictions = np.argmax(predictions, axis = 1)
y_true = np.argmax(y_test_ohe, axis =1)

In [22]:
cnt = 0
for i in range(len(y_true)):
  if y_true[i] == predictions[i]:
    cnt += 1

print("Accuarcy: ", 100 * cnt / len(y_true))

Accuarcy:  84.63873709775349
